# Import Librairie:

In [ ]:
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
from sqlalchemy import create_engine
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
import cv2
from pytesseract import Output
import pandas as pd
import tabula
import pdfplumber
from IPython.display import display

# Contexte:

Votre client souhaiterait extraire les tableaux de données des CRA (Compte-Rendu d'Activite).   
Ce type de document est complexe, il s'agit souvent de document format PDF de nombreuses pages, avec du texte mais également éléments des tableaux de chiffre qu'il serait intéressant de comparer pour faire des prédictions et évaluer les métriques par rapport aux attentes.  

​

Pour ce faire vous devrez utiliser les techniques d'OCR pour récupérer les données des fichiers PDF, puis de stocker ces données tabulaires dans un système de gestionnaire de bases de données relationnelles (par exemple MySQL).  

Le client souhaite que vous travaillez de manière itérative :  

dans un premier temps faites une veille pour tester la faisabalité des extractions de tableaux de chiffres d'un fichier PDF (le fichier data pourra vous être utile)  
puis l'extraction de différents tableaux dans une même page d'un PDF (le fichier test1 pourra être une bonne base).  
​

Bien sûr, pensez à automatiser au maximum votre code.  

## Fonctions

### Configuration SQL

Demande à l'utilisateur les configurations nécessaires pour se connecter à sa base de donnée SQL

In [ ]:
def config_sql():
    global hostname, dbname, uname, pwd
    print('Bonjour ! Veuiller entrer les paramètres de votre base de donnée SQL !')
    print('Veuillez entrer l\'hostname, si vous n\'entrer rien l\'hostname sera 127.0.0.1')
    hostname = str(input())
    if hostname == '':
        hostname = '127.0.0.1'
    print(hostname)
    print('Veuillez entrer le nom de la database (elle doit être déjà crée !)')
    dbname = str(input())
    print('Veuillez entrer votre username, si vous n\'entrer rien "root" sera rentré')
    uname = str(input())
    if uname == '':
        uname = 'root'
    print(uname)
    print('Veuillez entrer votre mot de passe SQL, si vous n\'entrer rien "root" sera rentré')
    pwd = str(input())
    if pwd == '':
        pwd  = 'root'

### Créations des tables SQL

Permet de se connecter à la base de donnée, et rentre toutes les informations dans une table que l'utilisateur peut donner le nom

In [ ]:
def sql_table():
    global hostname, dbname, uname, pwd, df_new
    print('Veuillez écrire le nom de la table')
    table = str(input())
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))
    df_new.to_sql(table, engine, index=False)

### Fonction transpose

Car le tableau peut avoir été mal lu et a besoin d'être transposé, cette fonction affiche les deux choix de sens de lecture possible. L'utilisateur chosit le tableau qu'il préfère garder, et celui-ci est stocké

In [ ]:
def transpose():
    global df, df_new
    restart = True
    while restart:
        restart = False
        print('Quel version préférez vous ?')
        df_1 = df
        df_2 = df.transpose()
        print('Ecrivez le numéro 1 pour ce tableau')
        display(df_1)
        print('Ecriver le numéro 2 pour ce tableau')
        display(df_2)
        print('')
        choix = int(input())
        if choix == 1:
            df_new = df_1
            print('Vous avez choisi le premier tableau')
        elif choix == 2:
            df_new = df_2
            print('Vous avez choisi le deuxième tableau')
        else:
            print('Veuillez entrer une bonne valeur')
            restart = True

### Fonction principale

Elle est la fonction que l'utilisateur doit lancer pour démarrer l'opération d'OCR de son fichier PDF

In [ ]:
def plumber():
    global pdf, pdf_file, df, df_new, hostname, dbname, uname, pwd
    config_sql()
    tampon = pdfplumber.open(pdf_file)
    for i in range(0,len(tampon.pages)):
        print('Voici la page'+str(i)+'')
        with pdfplumber.open(pdf_file) as pdf:
            for j in range(0,len(pdf.pages[i].find_tables())):
                pages = pdf.pages[i].find_tables()
                df = pages[j].extract()
                df = pd.DataFrame(df).replace({'\n': ''}, regex=True)
                transpose()
                new_header = df_new.iloc[0]
                df_new = df_new[1:]
                df_new.columns = new_header
                sql_table()
    print('Fin!')

# OCR 

Il vous suffit de rentrer le chemin du fichier pdf dont vous souhaiter récupérer les informations, puis delancer la fonction **plumber** pour effectuer le tout !

Une suite d'instructions vous sera affiché, vous serez guider tout du long

### Chemin du fichier :

Le chemin du fichier est nécessaire et doit être spécifié dans la cellule en dessous avant tout !

In [ ]:
pdf_file = (r"C:\Users\Simplon\Desktop\Travaux python\OCR\Text-Extraction-Table-Image-master\images\data.pdf")

### Démarrer l'OCR

In [ ]:
plumber()